In [7]:
import pandas as pd
import collab2.foraging.toolkit as ftk

import pyro
import pyro.distributions as dist
import pyro.optim as optim
import torch
from pyro.infer import SVI, Predictive, Trace_ELBO
from pyro.infer.autoguide import AutoMultivariateNormal, init_to_mean

In [8]:
# load data
fish_data = pd.read_csv("4wpf_test.csv")
gridMin = 0
gridMax = 300
grid_size = 50
fps = 121

# scaling and subsampling
fishDF_scaled = ftk.rescale_to_grid(
    fish_data, size=grid_size, gridMin=gridMin, gridMax=gridMax
)
fishDF_scaled_subsampled = ftk.subsample_frames_constant_frame_rate(
    fishDF_scaled, frame_spacing=121, fps=fps
)

# create a test foragers object with 20 frames
num_frames = 10
foragers_object = ftk.dataObject(
    fishDF_scaled_subsampled.loc[fishDF_scaled_subsampled["time"] < num_frames],
    grid_size=grid_size,
)

original_frames: 18739
original_shape: (74956, 5)
resulting_frames: 155
resulting_shape: (620, 5)
new frame-rate =  1.00


/Users/Admin/Repositories/collab-creatures/collab2/foraging/toolkit/utils.py:36: UserWarning:  Nan values in data.
                Specify handling of missing data using `skip_incomplete_frames` argument to `generate_all_predictors`
  warnings.warn(


In [9]:
# derive predictors

local_windows_kwargs = {
    "window_size": 15,
    "sampling_fraction": 1,
    "skip_incomplete_frames": True,
}

predictor_kwargs = {
    "vicsek": {
        "interaction_length": grid_size / 3,
        "interaction_constraint": None,
        "interaction_constraint_params": None,
        "dt": 1,
        "sigma_v": 3,
        "sigma_t": 0.3,
    },
    "vicsek_large_sigma": {
    "interaction_length": grid_size / 3,
    "interaction_constraint": None,
    "interaction_constraint_params": None,
    "dt": 1,
    "sigma_v": 10,
    "sigma_t": 1,
    },
    "pairwiseCopying": {
        "interaction_length": grid_size / 3,
        "interaction_constraint": None,
        "interaction_constraint_params": None,
        "dt": 1,
        "sigma_v": 3,
        "sigma_t": 0.3,
    },
}

combined_predictorDF = ftk.derive_predictors(
    foragers_object, local_windows_kwargs, predictor_kwargs, dropna=True, add_scaled_scores=True
)

/Users/Admin/Repositories/collab-creatures/collab2/foraging/toolkit/velocity.py:182: UserWarning: Dropped 7/36 instances from vicsek predictor calculation
            due to invalid velocity values
  warnings.warn(
2024-08-29 12:00:00,954 - vicsek completed
/Users/Admin/Repositories/collab-creatures/collab2/foraging/toolkit/velocity.py:32: UserWarning: Using existing velocity data.
                Delete corresponding columns from foragersDF to re-calculate velocity values.
  warnings.warn(
/Users/Admin/Repositories/collab-creatures/collab2/foraging/toolkit/velocity.py:182: UserWarning: Dropped 7/36 instances from vicsek_large_sigma predictor calculation
            due to invalid velocity values
  warnings.warn(
2024-08-29 12:00:01,047 - vicsek_large_sigma completed
/Users/Admin/Repositories/collab-creatures/collab2/foraging/toolkit/velocity.py:32: UserWarning: Using existing velocity data.
                Delete corresponding columns from foragersDF to re-calculate velocity values.
 

In [10]:
# calculate next_step_score

next_step_score = ftk.generate_next_step_score(foragers_object, nonlinearity_exponent=1)
combined_predictorDF.

# Try inference pipeline

In [ ]:
# convert each predictor and next_step_score to tensor